###           Le but de notebook est Essayer donner la bonne réponse à l'utilisateur à partir de database CSV.
 Première étape: Faire le traitement sur UNE PHRASE (comme ca on peut l'appliquer sur toute la base, ainsi que nouvelle question de l'utilisateur)
 
 
 Deuxième étape: Faire la vectorizer
 
 
 Troisième étape: Cosinus similarité.

In [35]:
import pandas as pd
import regex as re
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nlp = spacy.load('fr_core_news_sm')

In [2]:
dt = pd.read_csv("Questions_Rep.csv",sep = '\t',error_bad_lines=False)

In [3]:
dt

,Questions,Réponses
0,Que récolte le patch?,Le patch récolte plusieurs paramètres physiol...
1,Combien coûte le patch?,Il faut compter environ une cinquantaine d eu...
2,Où vont mes données?,Ils vont sur un serveur sécurisé pour permett...
3,Qui récolte mes données?,Les données sont récoltées sur un serveur séc...
4,Que fais-ce patch?,Il récolte des données physiologiques pour su...
5,Qui dirige Devinnova ?,Le directeur est Fabrice Vaussenat.
6,Qui est le directeur de Devinnova ?,Le directeur est Fabrice Vaussenat.
7,Qui puis-je appeler pour plus de renseignements ?,Voici le numéro du service communication : 061...
8,"Devinnova, c est quoi ?",Elle est une entreprise qui souhaite mettre l...
9,Pourquoi récolter mes données ?,Pour pouvoir traiter l activité de votre coeu...


### 2) Mettez le sous la forme de liste de phrases (pour le TFIDF)

In [4]:
def Put_to_List(data):
    l = []
    for el in data:
        l.append(el)
    return l
Questions = Put_to_List(dt['Questions'])
Réponses = Put_to_List(dt['Réponses'])

### Fonction pour  nettoyer le corpus


In [5]:
def Transform(phrase):
    phraseNLP = nlp(phrase)
    newPhrase = []
    for mot in phraseNLP:
        if not mot.is_punct and not mot.is_stop and not mot.is_space and len(mot)>1:
            newPhrase.append(mot.text)
    return " ".join(newPhrase)

### Appliquer le traitement sur toute Questions et Réponses en utilisant map

In [6]:
QuestTransf = map (Transform, Questions)
ReponTransf = map (Transform, Réponses)

In [7]:
dtTfidf = list(ReponTransf)

### Appliquer Vectorizer sur toute Questions/ Réponses ou les deux. Cad à la base de toute la liste, TFIDF va donner le poid de chaque mot dans l'ensemble de la liste. Ici je travaille sur la Réponse

In [32]:
vectorizer = TfidfVectorizer()
ReponsTfidf = vectorizer.fit_transform(dtTfidf)

In [31]:
ReponsTfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.26030005, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.2881334 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Cosinus similarité de toute la liste

In [34]:
cosine_similarity(ReponsTfidf)

array([[1.        , 0.        , 0.        , 0.        , 0.17721231,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.05278056, 0.05278056, 0.09135997, 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.78749184, 0.15670828,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.78749184, 1.        , 0.18156139,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.17721231, 0.        , 0.15670828, 0.18156139, 1.        ,
        0.        , 0.        , 

### Au cas ou l'utilisateur propose 1 nouvelle question

In [10]:
QuestUser = "Quand directeur Fabrice Vaussena est libre?"

1)  On met la nouvelle question dans la liste

In [11]:
dtTfidf.append(Transform(QuestUser))

In [12]:
print(dtTfidf)

['patch récolte paramètres physiologiques température corps saturation oxygène électrocardiogramme dérivations', 'faut compter euros', 'serveur sécurisé permettre médecin temps réel informations état santé utilisateur', 'données récoltées serveur sécurisé relié application médecin accès sécurisé permettre temps réel informations état santé utilisateur', 'récolte données physiologiques état cardiaque utilisateur traitement prédire potentiellement AVC', 'directeur Fabrice Vaussenat', 'directeur Fabrice Vaussenat', 'numéro service communication 0612536526', 'entreprise souhaite mettre patient coeur innovation est-à-dire qu puisse contribuer connaissance rétablissement faire commercialise dispositif myAngel VitalSigns', 'pouvoir traiter activité coeur identifier êtes susceptibles victimes AVC', 'électrocardiogramme tracé activité électrique coeur représentation graphique activité coeur Grâce tracé calcul connaître fréquence réspiratoire exemple', 'électrocardiogramme tracé activité électri

2) On applique le vectorizer sur toute la liste

In [13]:
vectorizer = TfidfVectorizer()
RepUTfidf = vectorizer.fit_transform(dtTfidf)

In [16]:
lenMatrix = RepUTfidf.toarray().shape[0]

3) On met la nouvelle question de l'utilisateur à part, et on fait consinus similarité entre la liste des réponses et la nouvelle question pour trouver la réponse la plus similaire de la réponse posée.

In [20]:
matrixCosSim = cosine_similarity(RepUTfidf[0:(lenMatrix-1)],RepUTfidf[-1,:])

In [21]:
matrixCosSim

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.48226191],
       [0.48226191],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

In [26]:
result = np.argmax(matrixCosSim)

In [29]:
Réponses[result]

'Le directeur est Fabrice Vaussenat.'